# Project

In [ ]:
# import catboost as cb
from collections import Counter
# from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
# from math import ceil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import shap
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, balanced_accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import mord
from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
# import xgboost as xgb
# from statsmodels.graphics.regressionplots import plot_regress_exog

In [ ]:
path = 'review-Alaska_10.json'

data_raw = pd.read_json(path, lines=True, encoding='utf-8')

This indicates the presence of NA values in the data. However, as this is one of the aspects we intend to investigate, we will selectively perform data cleaning at a later stage.

In [ ]:
print(data_raw.info())

In [ ]:
print(data_raw.head())

## Data Dictionary

- index: The index of the data.
- user_id: The ID of the reviewer.
- name: The name of the reviewer.
- time: The time of the review in Unix time format.
- rating: The rating given by the reviewer for the business.
- text: The text of the review.
- pics: Pictures associated with the review.
- resp: The business response to the review, including Unix time and the text of the response.
- gmap_id: The ID of the business.

Due to the nature of our research topic, which is to explore the influence of time of day on online ratings across different devices, we will be selecting specific data variables for further analysis. The data variables of interest include "time," "rating," and "pics." The reason for selecting "pics" is due to the unfortunate inability to obtain data directly related to device types in the comments. Therefore, we need to make a crucial assumption: 
**we assume that comments with pictures are uploaded using mobile devices, while comments without pictures are uploaded using non-mobile devices.**

## Exploratory Data Analysis (EDA)



In this section, we will perform data preprocessing, which includes data cleaning and data transformation. Data cleaning involves handling missing values, outliers, and inconsistencies in the dataset. Data transformation may involve converting the "pics" data into device type data, etc. These steps allow us to make use of the available information and derive meaningful insights from the dataset. 

In [ ]:
data_modified = (
    data_raw
    # Convert the timestamp to minutes since midnight
    .assign(minutes_since_midnight=lambda x: pd.to_datetime(x['time'], unit='ms', utc=True)
                                          .dt.tz_convert('America/Anchorage')
                                          .dt.hour * 60 
                                          + pd.to_datetime(x['time'], unit='ms', utc=True)
                                          .dt.tz_convert('America/Anchorage')
                                          .dt.minute)
    # device[0,1] represents ['Non-mobile devices', 'Mobile devices']
    .assign(device=lambda x: x['pics'].notnull().astype(int))
    # rating_binary[0,1] represents ['Rating not equal to 5', 'Rating equal to 5']
    .assign(rating_binary=lambda x: (x['rating'] == 5).astype(int))
    # length of text (words)
    .assign(text_length=lambda x: x['text'].apply(lambda t: len(t.split()) if t is not None else 0))
    # number of pics
    .assign(num_pics=lambda x: x['pics'].apply(lambda p: len(p) if isinstance(p, (list, pd.Series)) and p is not None else 0))
    # has_resp[0,1] represents ['No response', 'Has response']
    .assign(has_resp=lambda x: x['resp'].notnull().astype(int))
    .filter(['minutes_since_midnight', 'rating','rating_binary', 'device', 'text_length', 'num_pics', 'has_resp'])
).dropna()

print(data_modified[:10])


In [ ]:
data_modified.describe()

In [ ]:
# Hourly distribution
plt.figure(figsize=(10,6))
(data_modified['minutes_since_midnight'] // 60).value_counts().sort_index().plot(kind='bar', alpha=0.7)
plt.title('Distribution by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Count')
plt.show()

# Minute-by-minute distribution
plt.figure(figsize=(10,6))
data_modified['minutes_since_midnight'].value_counts().sort_index().plot(kind='bar', alpha=0.7)
plt.title('Distribution by Minute of Day')
plt.xlabel('Minute')
plt.ylabel('Count')

# Adjust x-axis ticks for better visibility
ticks = list(range(0, 1441, 120))  # Every 2 hours in minutes
labels = [f"{int(tick/60)}:{str(tick%60).zfill(2)}" for tick in ticks]  # Convert minute ticks to HH:MM format
plt.xticks(ticks, labels, rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Histogram for data distribution by rating
counts = data_modified['rating'].value_counts().sort_index()
counts.plot(kind='bar', alpha=0.7)
plt.xlabel('Rating')
plt.ylabel('Count')
plt.title('Distribution of Data by Rating')
plt.xticks(rotation=0)  # Rotate x-axis labels

for i, v in enumerate(counts):
    plt.text(i, v + 0.01 * counts.max(), f'{v / counts.sum() * 100:.1f}%', ha='center')

plt.show()

In [ ]:
# Histogram for data distribution by rating
counts = data_modified['rating_binary'].value_counts().sort_index()
counts.plot(kind='bar', alpha=0.7)
plt.xlabel('Rating')
plt.ylabel('Count')
plt.title('Distribution of Data by Rating (Binary)')
plt.xticks(rotation=0)  # Rotate x-axis labels

for i, v in enumerate(counts):
    plt.text(i, v + 0.01 * counts.max(), f'{v / counts.sum() * 100:.1f}%', ha='center')

plt.show()

In [ ]:
# Histogram for data distribution by devices
counts = data_modified['device'].value_counts().sort_index()
counts.plot(kind='bar', alpha=0.7)
plt.xlabel('Device')
plt.ylabel('Count')
plt.title('Distribution of Data by Device')
plt.xticks(rotation=0)  # Rotate x-axis labels

for i, v in enumerate(counts):
    plt.text(i, v + 0.01 * counts.max(), f'{v / counts.sum() * 100:.1f}%', ha='center')

plt.show()

In [ ]:
# Define the bin range for text_length
bins_text = list(range(0, 101, 5)) + [data_modified['text_length'].max() + 1]

# Labels for the bins
labels = [f"{bins_text[i]}-{bins_text[i+1]-1}" for i in range(len(bins_text)-2)] + ["100+"]

# Bin the text_length
data_modified['text_length_binned'] = pd.cut(data_modified['text_length'], bins=bins_text, right=False, labels=labels)

# Count the number of records in each bin
counts_text = data_modified['text_length_binned'].value_counts().sort_index()

# Plot
plt.figure(figsize=(15, 6))
counts_text.plot(kind='bar', alpha=0.7)
plt.xlabel('Text Length')
plt.ylabel('Count')
plt.title('Distribution of Data by Text Length')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.tight_layout()
plt.show()


In [ ]:
# Histogram for data distribution by num_pics
max_pics = 5  
data_modified['num_pics_binned'] = pd.cut(data_modified['num_pics'], bins=[-1, 1, 2, max_pics, data_modified['num_pics'].max()], labels=['0-1', '2', '3-'+str(max_pics), str(max_pics+1)+'+'])
counts_pics = data_modified['num_pics_binned'].value_counts().sort_index()
plt.figure(figsize=(12, 6))
counts_pics.plot(kind='bar', alpha=0.7)
plt.xlabel('Number of Pictures')
plt.ylabel('Count')
plt.title('Distribution of Data by Number of Pictures')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.tight_layout()
plt.show()

In [ ]:
# Histogram for data distribution by has_resp
counts = data_modified['has_resp'].value_counts().sort_index()
counts.plot(kind='bar', alpha=0.7)
plt.xlabel('Response')
plt.ylabel('Count')
plt.title('Distribution of Data by Response')
plt.xticks(rotation=0)  # Rotate x-axis labels

for i, v in enumerate(counts):
    plt.text(i, v + 0.01 * counts.max(), f'{v / counts.sum() * 100:.1f}%', ha='center')

plt.show()

In [ ]:
data_modified = data_modified.filter(['minutes_since_midnight', 'rating', 'rating_binary' , 'device', 'text_length', 'num_pics', 'has_resp'])
print(data_modified.isnull().sum())

### Relationships

In [ ]:
columns_to_plot = ['minutes_since_midnight', 'device', 'text_length', 'num_pics', 'has_resp']

# Create subplots for each column and for both 'rating' and 'rating_binary'
fig, axs = plt.subplots(len(columns_to_plot), 2, figsize=(15, 5*len(columns_to_plot)))

# For each column in the list
for i, col in enumerate(columns_to_plot):
    
    # Bar plot for 'rating'
    sns.barplot(x=col, y='rating', data=data_modified, ax=axs[i, 0], errorbar=None)
    axs[i, 0].set_title(f'Mean Rating by {col}')
    
    # Simplify x-axis labels if too many unique values
    if len(data_modified[col].unique()) > 20:
        ticks = axs[i, 0].get_xticks()
        visible_ticks = ticks[::5]  # Show every 5th tick
        axs[i, 0].set_xticks(visible_ticks)
        axs[i, 0].set_xticklabels(data_modified[col].unique()[::5])
    
    # Bar plot for 'rating_binary'
    sns.barplot(x=col, y='rating_binary', data=data_modified, ax=axs[i, 1], errorbar=None)
    axs[i, 1].set_title(f'Mean Rating Binary by {col}')
    
    # Simplify x-axis labels if too many unique values
    if len(data_modified[col].unique()) > 20:
        ticks = axs[i, 1].get_xticks()
        visible_ticks = ticks[::5]  # Show every 5th tick
        axs[i, 1].set_xticks(visible_ticks)
        axs[i, 1].set_xticklabels(data_modified[col].unique()[::5])

plt.tight_layout()
plt.show()


#### minutes_since_midnight and rating

In [ ]:
# Plotting the number of reviews and average rating per year
fig, ax1 = plt.subplots(figsize=(12, 6))

# Twin the axes for two different y-axes
ax2 = ax1.twinx()

# Convert the timestamp to a readable date format with correct timezone
date = pd.to_datetime(data_raw['time'], unit='ms', utc=True).dt.tz_convert('America/Anchorage')
year = date.dt.year

# Plotting the number of reviews per year
sns.countplot(x=year, ax=ax1, order=sorted(year.unique()), color='skyblue')

# Plotting the average rating per year
sns.pointplot(x=year, y=data_modified['rating'], ax=ax2, order=sorted(year.unique()), color='salmon', errorbar=None)

# Setting the y axis labels
ax1.set_ylabel('Number of Reviews', color='blue')
ax2.set_ylabel('Average Rating', color='red')

plt.title('Number of Reviews and Average Rating per Year')
plt.show()


In [ ]:
# Grouping by minutes_since_midnight and calculating the average rating
average_ratings = data_modified.groupby('minutes_since_midnight')['rating'].mean().reset_index()

# Plotting the average rating throughout the day
plt.figure(figsize=(12, 6))
sns.lineplot(data=average_ratings, x='minutes_since_midnight', y='rating', color="salmon")
plt.title('Average Rating Throughout the Day')
plt.xlabel('Minutes Since Midnight')
plt.ylabel('Average Rating')
plt.show()


#### device and rating

In [ ]:
# KDE plot showing distribution of ratings based on device type
plt.figure(figsize=(12, 6))
sns.kdeplot(data=data_modified[data_modified['device'] == 0]['rating'], label="Non-mobile devices", fill=True)
sns.kdeplot(data=data_modified[data_modified['device'] == 1]['rating'], label="Mobile devices", fill=True)
plt.title('Distribution of Ratings Based on Device Type')
plt.xlabel('Rating')
plt.ylabel('Density')
plt.legend()
plt.show()


#### text length and rating

In [ ]:
# Plotting the boxplot for text_length vs rating
plt.figure(figsize=(12, 6))
sns.boxplot(data=data_modified, x='rating', y='text_length', palette="pastel")
plt.title('Boxplot of Text Length vs Rating')
plt.ylim(0, 1000)  # Limiting y-axis for a clearer view
plt.show()


#### number of pics and rating

In [ ]:
# Plotting a boxplot to show the distribution of pic count for each rating
plt.figure(figsize=(12, 6))
data_modified.boxplot(column='num_pics', by='rating', grid=False, vert=False, patch_artist=True)
plt.title('Distribution of Picture Count by Rating')
plt.xlabel('Picture Count')
plt.ylabel('Rating')
plt.suptitle('')  # Suppress the default suptitle
plt.tight_layout()

plt.show()

#### has_resp and rating

In [ ]:
# Plotting the Count of reviews by rating and response status with rating on y-axis
plt.figure(figsize=(10, 6))
sns.countplot(data=data_modified, y='rating', hue='has_resp', palette="pastel")
plt.title('Count of Reviews by Rating and Response Status')
plt.ylabel('Rating')
plt.xlabel('Number of Reviews')
plt.legend(title='Has Response', labels=['No Response', 'Has Response'])
plt.show()

In [ ]:
# # Plotting the boxplot for has_resp vs rating
# plt.figure(figsize=(12, 6))
# sns.boxplot(data=data_modified, x='rating', y='has_resp', palette="pastel", orient="h")
# plt.title('Boxplot of Whether Merchant Responded vs Rating')
# plt.show()


### Pearson Correlation

In [ ]:
# Correlation analysis
correlation_matrix = data_modified.corr()

# Extract correlation with 'rating' and 'rating_binary'
correlation_with_rating = correlation_matrix[['rating', 'rating_binary']].drop(['rating', 'rating_binary'])

correlation_with_rating


### Chi-square

In [ ]:
# Identify categorical variables
categorical_vars = ['device', 'rating', 'rating_binary', 'has_resp']

def detailed_chi2_pvalues(data, cat_vars):
    results = []
    for var1 in cat_vars:
        for var2 in cat_vars:
            if var1 != var2:
                chi2_stat, p_val, dof, expected = stats.chi2_contingency(pd.crosstab(data[var1], data[var2]))
                results.append({
                    'Variable 1': var1,
                    'Variable 2': var2,
                    'Chi2 Statistic': chi2_stat,
                    'P-value': p_val,
                    'Degrees of Freedom': dof,
                    'Minimum Expected Frequency': expected.min()
                })
    return pd.DataFrame(results)

detailed_chi2_df = detailed_chi2_pvalues(data_modified, categorical_vars)
detailed_chi2_df


### Spearman Correlation

In [ ]:
# Identify continuous variables
continuous_vars = ['minutes_since_midnight', 'text_length', 'num_pics']

def detailed_spearman_correlation(data, cont_vars):
    results = []
    for var1 in cont_vars:
        for var2 in cont_vars:
            if var1 != var2:
                corr, p_val = stats.spearmanr(data[var1], data[var2])
                results.append({
                    'Variable 1': var1,
                    'Variable 2': var2,
                    'Spearman Correlation': corr,
                    'P-value': p_val
                })
    return pd.DataFrame(results)

detailed_spearman_df = detailed_spearman_correlation(data_modified, continuous_vars)
detailed_spearman_df


### Multicollinearity test

In [ ]:
X = data_modified[['minutes_since_midnight', 'rating', 'rating_binary', 'device', 'text_length', 'num_pics', 'has_resp']]
vif_data = pd.DataFrame()
vif_data['Features'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("\nMulticollinearity test:")
print(vif_data)

## Modeling

### Train and Test Sets

In [ ]:
# Separate features (X) and target variable (y) in the balanced dataset
X = data_modified.drop(['rating', 'rating_binary'], axis=1)
y = data_modified['rating']

X_bi = data_modified.drop(['rating', 'rating_binary'], axis=1)
y_bi = data_modified['rating_binary']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2221877)


In [ ]:
# Split indices
train_idx, test_idx = train_test_split(data_modified.index, test_size=0.2, random_state=2221877)

# Use indices to extract training and testing data
X_train = data_modified.loc[train_idx].drop(['rating', 'rating_binary'], axis=1)
y_train = data_modified.loc[train_idx]['rating']

X_train_bi = data_modified.loc[train_idx].drop('rating', axis=1)
y_train_bi = data_modified.loc[train_idx]['rating_binary']

X_test = data_modified.loc[test_idx].drop(['rating', 'rating_binary'], axis=1)
y_test = data_modified.loc[test_idx]['rating']

X_test_bi = data_modified.loc[test_idx].drop('rating', axis=1)
y_test_bi = data_modified.loc[test_idx]['rating_binary']


### Data Balancing

It can be seen that there is a significant data imbalance between the different ratings. Having tried sampling and undersampling, I ended up combining them to try and get the most optimal sampling results to support my models.

In [ ]:
# Calculate the number of samples in each class
counter = Counter(y)
counter_bi = Counter(y_bi)

# Define the target sample numbers for over-sampling and under-sampling
max_samples = max(counter.values())
min_samples = min(counter.values())
max_samples_bi = max(counter_bi.values())
min_samples_bi = min(counter_bi.values())

# Define pipeline
over = SMOTE(sampling_strategy={class_label: max_samples for class_label in counter})
under = RandomUnderSampler(sampling_strategy={class_label: min_samples for class_label in counter})
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)

over_bi = SMOTE(sampling_strategy={class_label: max_samples_bi for class_label in counter_bi})
under_bi = RandomUnderSampler(sampling_strategy={class_label: min_samples_bi for class_label in counter_bi})
steps_bi = [('o', over_bi), ('u', under_bi)]
pipeline_bi = Pipeline(steps=steps_bi)

# Apply the pipeline
X_resampled, y_resampled = pipeline.fit_resample(X, y)
X_resampled_bi, y_resampled_bi = pipeline_bi.fit_resample(X_bi, y_bi)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=2221877)
X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(X_resampled_bi, y_resampled_bi, test_size=0.2, random_state=2221877)


In [ ]:
# Count samples for each class in 'rating' and 'rating_binary'
count_rating = Counter(y_resampled)
count_rating_bi = Counter(y_resampled_bi)

count_rating, count_rating_bi

### Decision Tree

In [ ]:
model_DT = DecisionTreeClassifier()

model_DT.fit(X_resampled, y_resampled)

predictions_DT = model_DT.predict(X_test)

print('Accuracy score: ', balanced_accuracy_score(y_test, predictions_DT))
print(classification_report(y_test, predictions_DT))

cm = confusion_matrix(y_test, predictions_DT)
sns.heatmap(cm, annot=True, fmt=".0f")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Create an Explainer object
explainer_DT = shap.Explainer(model_DT)

# Calculate SHAP values
shap_values_DT = explainer_DT.shap_values(X_test)

# Visualise the first prediction's explanation
shap.summary_plot(shap_values_DT, X_test, plot_type="bar")

In [ ]:
def plot_multiclass_summary(shap_values_list, X_test, class_names=None, max_display=None):
    num_classes = len(shap_values_list)
    
    if max_display is None:
        max_display = num_classes
    
    if class_names is None:
        class_names = [f"Class {i}" for i in range(num_classes)]
    
    for i in range(max_display):
        shap_values_class = shap_values_list[i]
        class_name = class_names[i]
        
        # Display summary plot for the class
        shap.summary_plot(shap_values_class, X_test, show=False, plot_type='dot', title=class_name)
        plt.title(class_name)
        plt.show()
        
        # Print average SHAP values for each feature for the class
        print(f"\nAverage SHAP values for {class_name}:\n")
        shap_avg = np.abs(shap_values_class).mean(axis=0)
        for feature, value in zip(X_test.columns, shap_avg):
            print(f"{feature}: {value:.4f}")
        print("\n" + "="*50 + "\n")

In [ ]:
# Call the function with shap_values_DT and X_test
plot_multiclass_summary(shap_values_DT, X_test)

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

# Using the Decision Tree model as an example, and targeting class 1
features = ['minutes_since_midnight', 'device', 'text_length', 'num_pics', 'has_resp']

# Create a larger figure
fig, ax = plt.subplots(len(features), 1, figsize=(8, len(features) * 4))

display = PartialDependenceDisplay.from_estimator(model_DT, X_test, features, target=1, ax=ax)

# Adjust layout for better visibility
fig.tight_layout(pad=3.0)

# Display the plots
plt.show()



In [ ]:
# Compute SHAP interaction values
shap_interaction_values = explainer_DT.shap_interaction_values(X_test)  # using a subset for visualization

# Just using the interaction values for the first class as an example
shap_values_class_0 = shap_interaction_values[0]

# Plot
shap.dependence_plot(
    ("minutes_since_midnight", "text_length"), 
    shap_values_class_0,
    X_test
)

In [ ]:
shap_interaction_values = explainer_DT.shap_interaction_values(X_test)

# Plotting interaction values for all features
shap.summary_plot(shap_interaction_values, X_test)


In [ ]:
model_DT_bi = DecisionTreeClassifier()

model_DT_bi.fit(X_resampled_bi, y_resampled_bi)

predictions_DT_bi = model_DT_bi.predict(X_test_bi)

print('Accuracy score: ', balanced_accuracy_score(y_test_bi, predictions_DT_bi))
print(classification_report(y_test_bi, predictions_DT_bi))

cm_bi = confusion_matrix(y_test_bi, predictions_DT_bi)
sns.heatmap(cm_bi, annot=True, fmt=".0f")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
# Create an Explainer object
explainer_DT_bi = shap.Explainer(model_DT_bi)

# Calculate SHAP values
shap_values_DT_bi = explainer_DT_bi.shap_values(X_test_bi)

# Visualise the first prediction's explanation
shap.summary_plot(shap_values_DT_bi, X_test_bi)

In [ ]:
# Call the function with shap_values_DT_bi and X_test_bi
plot_multiclass_summary(shap_values_DT_bi, X_test_bi)

### Random Forest

In [ ]:
model_RF = RandomForestClassifier()

model_RF.fit(X_resampled, y_resampled)

predictions_RF = model_RF.predict(X_test)

print('Accuracy score: ', balanced_accuracy_score(y_test, predictions_RF))
print(classification_report(y_test, predictions_RF))

cm = confusion_matrix(y_test, predictions_RF)
sns.heatmap(cm, annot=True, fmt=".0f")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
model_RF_bi = RandomForestClassifier()

model_RF_bi.fit(X_resampled_bi, y_resampled_bi)

predictions_RF_bi = model_RF_bi.predict(X_test_bi)

print('Accuracy score: ', balanced_accuracy_score(y_test_bi, predictions_RF_bi))
print(classification_report(y_test_bi, predictions_RF_bi))

cm_bi = confusion_matrix(y_test_bi, predictions_RF_bi)
sns.heatmap(cm_bi, annot=True, fmt=".0f")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

### Ordinal Logistic Regression

In [ ]:
# Create an instance of the LogisticAT model
model_OL = mord.LogisticAT()

# Fit the model to the data
model_OL.fit(X_resampled, y_resampled)

# Use the model to make predictions
predictions_OL = model_OL.predict(X_test)

# Evaluate the model's performance
print('Accuracy score: ', balanced_accuracy_score(y_test, predictions_OL))
print(classification_report(y_test, predictions_OL))

# Plot the confusion matrix
cm = confusion_matrix(y_test, predictions_OL)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Create an instance of the LogisticAT model
model_OL_bi = mord.LogisticAT()

# Fit the model to the data
model_OL_bi.fit(X_resampled_bi, y_resampled_bi)

# Use the model to make predictions
predictions_OL_bi = model_OL_bi.predict(X_test_bi)

# Evaluate the model's performance
print('Accuracy score: ', balanced_accuracy_score(y_test_bi, predictions_OL_bi))
print(classification_report(y_test_bi, predictions_OL_bi))

# Plot the confusion matrix
cm_bi = confusion_matrix(y_test_bi, predictions_OL_bi)
plt.figure(figsize=(10,7))
sns.heatmap(cm_bi, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


### Gradient Boosting 

In [ ]:
# Create a GradientBoostingClassifier
model_GB = GradientBoostingClassifier()

model_GB.fit(X_resampled, y_resampled)

predictions_GB = model_GB.predict(X_test)

# Evaluate the model's performance
print('Accuracy score: ', balanced_accuracy_score(y_test, predictions_GB))
print(classification_report(y_test, predictions_GB))

# Plot the confusion matrix
cm = confusion_matrix(y_test, predictions_GB)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Create a GradientBoostingClassifier
model_GB_bi = GradientBoostingClassifier()

model_GB_bi.fit(X_resampled_bi, y_resampled_bi)

predictions_GB_bi = model_GB_bi.predict(X_test_bi)

# Evaluate the model's performance
print('Accuracy score: ', balanced_accuracy_score(y_test_bi, predictions_GB_bi))
print(classification_report(y_test_bi, predictions_GB_bi))

# Plot the confusion matrix
cm_bi = confusion_matrix(y_test_bi, predictions_GB_bi)
plt.figure(figsize=(10,7))
sns.heatmap(cm_bi, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


### XGBoost

In [ ]:
# Create an XGBoost classifier
model_XGB = xgb.XGBClassifier()

# Adjust labels for XGBoost
y_resampled_adjusted = y_resampled - 1
y_test_adjusted = y_test - 1

# Fit the model to the data
model_XGB.fit(X_resampled, y_resampled_adjusted)

# Make predictions
predictions_XGB = model_XGB.predict(X_test)

# Revert the predictions to the original scale
predictions_XGB = predictions_XGB + 1

# Evaluate the model's performance
print('Accuracy score (XGBoost): ', balanced_accuracy_score(y_test, predictions_XGB))
print(classification_report(y_test, predictions_XGB))

# Plot the confusion matrix
cm_XGB = confusion_matrix(y_test, predictions_XGB)
plt.figure(figsize=(10,7))
sns.heatmap(cm_XGB, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (XGBoost)')
plt.show()


In [ ]:
# Get feature importance
feature_importance = model_XGB.feature_importances_

# Create a mapping of feature names and their importance
feature_importance_dict = dict(zip(X.columns, feature_importance))

# Sort by importance
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Display sorted feature importance
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance:.4f}")

# Visualize feature importance
plt.figure(figsize=(10, 8))
xgb.plot_importance(model_XGB, importance_type='weight')
plt.title('Feature Importance')
plt.show()


In [ ]:
# Create an XGBoost classifier
model_XGB_bi = xgb.XGBClassifier()

# Fit the model to the data
model_XGB_bi.fit(X_resampled_bi, y_resampled_bi)

# Make predictions
predictions_XGB_bi = model_XGB_bi.predict(X_test_bi)

# Evaluate the model's performance
print('Accuracy score (XGBoost): ', balanced_accuracy_score(y_test_bi, predictions_XGB_bi))
print(classification_report(y_test_bi, predictions_XGB_bi))

# Plot the confusion matrix
cm_XGB_bi = confusion_matrix(y_test_bi, predictions_XGB_bi)
plt.figure(figsize=(10,7))
sns.heatmap(cm_XGB_bi, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (XGBoost)')
plt.show()


In [ ]:
# Get feature importance for binary classification
feature_importance_bi = model_XGB_bi.feature_importances_

# Create a mapping of feature names and their importance for binary classification
feature_importance_dict_bi = dict(zip(X_resampled_bi.columns, feature_importance_bi))

# Sort by importance for binary classification
sorted_feature_importance_bi = sorted(feature_importance_dict_bi.items(), key=lambda x: x[1], reverse=True)

# Display sorted feature importance for binary classification
for feature, importance in sorted_feature_importance_bi:
    print(f"{feature}: {importance:.4f}")

# Visualize feature importance for binary classification
plt.figure(figsize=(10, 8))
xgb.plot_importance(model_XGB_bi, importance_type='weight')
plt.title('Feature Importance (Binary Classification)')
plt.show()

### CatBoost

In [ ]:
# Identify categorical features
cat_features = ['device', 'has_resp']

# Create a CatBoost classifier
model_CB = cb.CatBoostClassifier(cat_features=cat_features, verbose=0)

# Fit the model to the data
model_CB.fit(X_resampled, y_resampled)

# Make predictions
predictions_CB = model_CB.predict(X_test)

# Evaluate the model's performance
print('Accuracy score: ', balanced_accuracy_score(y_test, predictions_CB))
print(classification_report(y_test, predictions_CB))

# Plot the confusion matrix
cm_CB = confusion_matrix(y_test, predictions_CB)
plt.figure(figsize=(10,7))
sns.heatmap(cm_CB, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (CatBoost)')
plt.show()


In [ ]:
# Create a CatBoost classifier
model_CB_bi = cb.CatBoostClassifier(cat_features=cat_features, verbose=0)

# Fit the model to the data
model_CB_bi.fit(X_resampled_bi, y_resampled_bi)

# Make predictions
predictions_CB_bi = model_CB_bi.predict(X_test_bi)

# Evaluate the model's performance
print('Accuracy score: ', balanced_accuracy_score(y_test_bi, predictions_CB_bi))
print(classification_report(y_test_bi, predictions_CB_bi))

# Plot the confusion matrix
cm_CB_bi = confusion_matrix(y_test_bi, predictions_CB_bi)
plt.figure(figsize=(10,7))
sns.heatmap(cm_CB_bi, annot=True, fmt=".0f", cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (CatBoost)')
plt.show()


### Model Comparisons


In [ ]:
# Define a function to plot ROC curves for each class in a subplot
def plot_multiclass_roc_auc_subplot(ax, model, X_test, y_test, class_idx, model_name, lw=2):
    y_score = model.predict_proba(X_test)
    fpr, tpr, _ = roc_curve(y_test == class_idx + 1, y_score[:, class_idx])
    roc_auc = auc(fpr, tpr)
    ax.plot(fpr, tpr, lw=lw)
    ax.set_title(f'{model_name} - Class {class_idx + 1}')
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    return roc_auc

# Initialize subplots
fig, axes = plt.subplots(len(models), 5, figsize=(25, 25)) # Adjust figsize

auc_results = {}

# Loop through each model and plot ROC curves in subplots
models = [model_DT, model_RF, model_GB, model_OL, model_XGB, model_CB]
model_names = ['Decision Tree', 'Random Forest', 'Gradient Boosting', 'Ordered Logit', 'XGBoost', 'CatBoost']

for idx, model in enumerate(models):
    avg_aucs = []
    for i, ax in enumerate(axes[idx]):
        auc_score = plot_multiclass_roc_auc_subplot(ax, model, X_test, y_test, i, model_names[idx])
        avg_aucs.append(auc_score)
    avg_auc = np.mean(avg_aucs)
    auc_results[model_names[idx]] = avg_auc

# Adjust layout
plt.tight_layout()
plt.show()

# Display overall average AUC results
for model_name, auc_score in auc_results.items():
    print(f"{model_name} Overall Average AUC: {auc_score:.2f}")


In [ ]:
# Plot overall average AUC results
plt.figure(figsize=(10, 6))
model_names = list(auc_results.keys())
avg_aucs = list(auc_results.values())

# Create a bar plot for the AUC scores of each model
sns.barplot(x=model_names, y=avg_aucs, palette="viridis")
plt.xlabel('Model')
plt.ylabel('Overall Average AUC')
plt.title('Model AUC Comparison')
plt.ylim([0, 1])  # Set y-axis limits to [0, 1] for AUC
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()

# Display the actual AUC values on top of each bar
for i, v in enumerate(avg_aucs):
    plt.text(i, v + 0.02, f"{v:.2f}", ha='center', va='bottom', fontweight='bold')

plt.show()


In [ ]:
# Predict probabilities for all models
probs_DT = model_DT_bi.predict_proba(X_test_bi)[:, 1]
probs_RF = model_RF_bi.predict_proba(X_test_bi)[:, 1]
probs_GB = model_GB_bi.predict_proba(X_test_bi)[:, 1]
probs_OL = model_OL_bi.predict_proba(X_test_bi)[:, 1]
probs_XGB = model_XGB_bi.predict_proba(X_test_bi)[:, 1]
probs_CB = model_CB_bi.predict_proba(X_test_bi)[:, 1]

# Compute ROC curve and ROC area for each model
fpr_DT, tpr_DT, _ = roc_curve(y_test_bi, probs_DT)
roc_auc_DT = auc(fpr_DT, tpr_DT)

fpr_RF, tpr_RF, _ = roc_curve(y_test_bi, probs_RF)
roc_auc_RF = auc(fpr_RF, tpr_RF)

fpr_GB, tpr_GB, _ = roc_curve(y_test_bi, probs_GB)
roc_auc_GB = auc(fpr_GB, tpr_GB)

fpr_OL, tpr_OL, _ = roc_curve(y_test_bi, probs_OL)
roc_auc_OL = auc(fpr_OL, tpr_OL)

fpr_XGB, tpr_XGB, _ = roc_curve(y_test_bi, probs_XGB)
roc_auc_XGB = auc(fpr_XGB, tpr_XGB)

fpr_CB, tpr_CB, _ = roc_curve(y_test_bi, probs_CB)
roc_auc_CB = auc(fpr_CB, tpr_CB)

# Plot ROC curves in the same plot
plt.figure(figsize=(10, 8))
lw = 2
plt.plot(fpr_DT, tpr_DT, color='darkorange', lw=lw, label=f'Decision Tree (AUC = {roc_auc_DT:0.2f})')
plt.plot(fpr_RF, tpr_RF, color='blue', lw=lw, label=f'Random Forest (AUC = {roc_auc_RF:0.2f})')
plt.plot(fpr_GB, tpr_GB, color='green', lw=lw, label=f'Gradient Boosting (AUC = {roc_auc_GB:0.2f})')
plt.plot(fpr_OL, tpr_OL, color='red', lw=lw, label=f'Ordered Logit (AUC = {roc_auc_OL:0.2f})')
plt.plot(fpr_XGB, tpr_XGB, color='purple', lw=lw, label=f'XGBoost (AUC = {roc_auc_XGB:0.2f})')
plt.plot(fpr_CB, tpr_CB, color='cyan', lw=lw, label=f'CatBoost (AUC = {roc_auc_CB:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

# Optional: Print out AUC scores
print("Decision Tree AUC:", roc_auc_score(y_test_bi, probs_DT))
print("Random Forest AUC:", roc_auc_score(y_test_bi, probs_RF))
print("Gradient Boosting AUC:", roc_auc_score(y_test_bi, probs_GB))
print("Ordered Logit AUC:", roc_auc_score(y_test_bi, probs_OL))
print("XGBoost AUC:", roc_auc_score(y_test_bi, probs_XGB))
print("CatBoost AUC:", roc_auc_score(y_test_bi, probs_CB))
